In [2]:
#A nested dictionary is created which shows individual rating for movies given by each user 

critics={
'Rohit Jain': {'Avengers Age of Ultron': 2.5, 'Batman vs. Superman': 3.5, 'Knight and Day': 3.0, 'Superman Returns': 3.5, 'Casino Royale': 2.5, 'The Amazing Spiderman': 3.0}, 
'Ruchika Mittal': {'Avengers Age of Ultron': 3.0, 'Batman vs. Superman': 3.5, 'Knight and Day': 1.5, 'Superman Returns': 5.0, 'The Amazing Spiderman': 3.0, 'Casino Royale': 3.5},
 'Arpit Joshi': {'Avengers Age of Ultron': 2.5, 'Batman vs. Superman': 3.0, 'Superman Returns': 3.5, 'The Amazing Spiderman': 4.0}, 
'Tanya Mathur': {'Batman vs. Superman': 3.5, 'Knight and Day': 3.0, 'The Amazing Spiderman': 4.5, 'Superman Returns': 4.0, 'Casino Royale': 2.5},
 'Tanvi Jain': {'Avengers Age of Ultron': 3.0, 'Batman vs. Superman': 4.0, 'Knight and Day': 2.0, 'Superman Returns': 3.0, 'The Amazing Spiderman': 3.0, 'Casino Royale': 2.0},
 'Harsha Gupta': {'Avengers Age of Ultron': 3.0, 'Batman vs. Superman': 4.0, 'The Amazing Spiderman': 3.0, 'Superman Returns': 5.0, 'Casino Royale': 3.5},
 'Yash Singh': {'Batman vs. Superman':4.5,'Casino Royale':1.0,'Superman Returns':4.0}
}


In [5]:
# algo for Euclidean similarity

from math import pow

# returns a distance based similarity score for person1 and person2
def similarity_distance(prefs,person1,person2):
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
 #if they have no ratings in common,we will return 0

    if len(si)==0:
        return 0

 #add up the squares of all the difference
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2)for item in prefs[person1]if item in prefs[person2]])

    return 1/(1+sum_of_squares)

similarity_distance(critics, 'Rohit Jain', 'Ruchika Mittal')


0.14814814814814814

In [7]:
#algo for Pearson similarity
from math import sqrt

def sim_pearson(prefs,p1,p2):
 # Get the list of mutually rated items
 si={}
 for item in prefs[p1]:
    if item in prefs[p2]: si[item]=1
 # Find the number of elements
 n=len(si)
 # if they are no ratings in common, return 0
 if n==0: return 0
 # Add up all the preferences
 sum1=sum([prefs[p1][it] for it in si])
 sum2=sum([prefs[p2][it] for it in si])
 # Sum up the squares
 sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
 sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
 # Sum up the products
 pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
 # Calculate Pearson score
 num=pSum-(sum1*sum2/n)
 den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
 if den==0: return 0
 r=num/den
 return r

print(sim_pearson(critics,'Rohit Jain','Ruchika Mittal'))



0.39605901719066977


In [8]:
#Ranking the critics
def topMatches(prefs,person,n=5,similarity=sim_pearson):
 scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
 # Sort the list so the highest scores appear at the top
 scores.sort( )
 scores.reverse( )
 return scores[0:n]

print(topMatches(critics,'Yash Singh',n=3))


[(0.9912407071619299, 'Rohit Jain'), (0.9244734516419049, 'Tanvi Jain'), (0.8934051474415647, 'Tanya Mathur')]


In [9]:
#User Based Filtering Method to find recommendations
def getRecommendations(preference,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in preference:
        if other==person: continue
        sim=similarity(preference,person,other)

        if sim<=0: continue
        for item in preference[other]:
            
            if item not in preference[person] or preference[person][item]==0:
                
                totals.setdefault(item,0)
                totals[item]+=preference[other][item]*sim
                
                simSums.setdefault(item,0)
                simSums[item]+=sim
   
    rankings=[(total/simSums[item],item) for item,total in totals.items()]

    
    rankings.sort()
    rankings.reverse()
    return rankings
#get recommended item for Yash Singh
print(getRecommendations(critics,'Yash Singh'))


[(3.3477895267131017, 'The Amazing Spiderman'), (2.8325499182641614, 'Avengers Age of Ultron'), (2.530980703765565, 'Knight and Day')]


In [17]:
#Item Based Method

#Matching Products
def transformPrefs(preference):
    result={}
    for person in preference:
        for  item in preference[person]:
            result.setdefault(item,{})

            #flip item and person
            result[item][person]=preference[person][item]
    return result

print(transformPrefs(critics))

#Item Comparison Dataset
def calculateSimilarItems(prefs,n=10):
    #create a dictionary of items showing which other items they are similar to
    result={}
    #invert the preference matrix to be item centric
    itemPrefs=transformPrefs(prefs)
    c=0
    for item in itemPrefs:
        #status updates for large datasets
        c+=1
        if c%100==0:
            print ("%d / %d" % (c,len(itemPrefs)))
        #Find the most similar items to this one
        scores=topMatches(itemPrefs,item,n=n,similarity=similarity_distance)
        result[item]=scores
    return result
print('\n')
itemsimilarity=calculateSimilarItems(critics)
print(itemsimilarity) 

#Item Based Recommendation
def getRecommendedItems(prefs,itemMatch,user):
    userRatings=prefs[user]
    scores={}
    totalSim={}

    #loop over item rated by this user
    for (item,rating) in userRatings.items():

        #loop over items similar to this one
        for (similarity,item2) in itemMatch[item]:

            #Ignore if this user has already rated this item
            if item2 in userRatings: continue

            #weighted sum of rating times similarity
            scores.setdefault(item2,0)
            scores[item2]+=similarity*rating

            #Sum of all the similarities
            totalSim.setdefault(item2,0)
            totalSim[item2]+=similarity

    # Divide each total score by total weighting to get an average
    rankings = [(score / totalSim[item], item) for item, score in scores.items()]

    # Return the rankings from highest to lowest
    rankings.sort()
    rankings.reverse()
    return rankings
print('\n')
#get recommended items for Arpit
print(getRecommendedItems(critics,itemsimilarity,'Arpit Joshi') )



{'Avengers Age of Ultron': {'Rohit Jain': 2.5, 'Ruchika Mittal': 3.0, 'Arpit Joshi': 2.5, 'Tanvi Jain': 3.0, 'Harsha Gupta': 3.0}, 'Batman vs. Superman': {'Rohit Jain': 3.5, 'Ruchika Mittal': 3.5, 'Arpit Joshi': 3.0, 'Tanya Mathur': 3.5, 'Tanvi Jain': 4.0, 'Harsha Gupta': 4.0, 'Yash Singh': 4.5}, 'Knight and Day': {'Rohit Jain': 3.0, 'Ruchika Mittal': 1.5, 'Tanya Mathur': 3.0, 'Tanvi Jain': 2.0}, 'Superman Returns': {'Rohit Jain': 3.5, 'Ruchika Mittal': 5.0, 'Arpit Joshi': 3.5, 'Tanya Mathur': 4.0, 'Tanvi Jain': 3.0, 'Harsha Gupta': 5.0, 'Yash Singh': 4.0}, 'Casino Royale': {'Rohit Jain': 2.5, 'Ruchika Mittal': 3.5, 'Tanya Mathur': 2.5, 'Tanvi Jain': 2.0, 'Harsha Gupta': 3.5, 'Yash Singh': 1.0}, 'The Amazing Spiderman': {'Rohit Jain': 3.0, 'Ruchika Mittal': 3.0, 'Arpit Joshi': 4.0, 'Tanya Mathur': 4.5, 'Tanvi Jain': 3.0, 'Harsha Gupta': 3.0}}


{'Avengers Age of Ultron': [(0.4, 'Casino Royale'), (0.2857142857142857, 'The Amazing Spiderman'), (0.2222222222222222, 'Knight and Day'), (0.2